In [15]:
import subprocess
import shlex
import pandas as pd
import numpy as np
from astropy.table import Table
from astropy.table import Column
import os
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import MultipleLocator
import glob
from matplotlib import pyplot
import matplotlib.gridspec as gridspec
import gc
import astropy
from scipy import interpolate
import pickle
from astropy.time import Time

In [40]:
basedir = "/home/xhall/Documents/"
source = basedir + "NewZTF/sample_2020/SNIDoutput/"
output = basedir + "NewZTF/sample_2020/BTSoutput/"

sample200 = Table.from_pandas(pd.read_csv(basedir + "NewZTF/ML_sample_snid200.csv"))
ML_sample_snid_examples = Table.from_pandas(pd.read_csv(basedir + "NewZTF/ML_sample_snid_brightexamples.csv"))
sample_2018 = Table.from_pandas(pd.read_csv(basedir + "NewZTF/sample_2018/ML_sample_snid_norepeat_2018.csv"))
sample_2020 = Table.from_pandas(pd.read_csv("/home/xhall/Documents/NewZTF/sample_2020/SNID_2020_output.csv"))
sample_remaining = Table.from_pandas(pd.read_csv("/home/xhall/Documents/ZTFI/SNID_2020_remaining_output.csv"))
superfit_results = Table.from_pandas(pd.read_csv("/home/xhall/Documents/Superfit/superfit_classes_ZTFI.csv"))
a_file = open("/home/xhall/Documents/Superfit/short_path_dict.pkl", "rb")
short_path_dict = pickle.load(a_file)
phase_data = Table.from_pandas(pd.read_csv("/home/xhall/Documents/ZTFI/phase.csv"))

In [41]:
joined_table = astropy.table.join(sample_2020, superfit_results)

In [44]:
joined_table[0]

Version,ZTF_Name,z_sntemplate,z_rlap,z_snid,z_snid_err,z_level,rank_1,sntemplate_1,rlap_1,c_snid_1,z_snid_1,z_snid_err_1,age_1,age_flag_1,rank_2,sntemplate_2,rlap_2,c_snid_2,z_snid_2,z_snid_err_2,age_2,age_flag_2,rank_3,sntemplate_3,rlap_3,c_snid_3,z_snid_3,z_snid_err_3,age_3,age_flag_3,rank_4,sntemplate_4,rlap_4,c_snid_4,z_snid_4,z_snid_err_4,age_4,age_flag_4,rank_5,sntemplate_5,rlap_5,c_snid_5,z_snid_5,z_snid_err_5,age_5,age_flag_5,Resolution,Superfit_c,Superfit_z,Match,Gal,Frac(SN),Frac(gal),Phase,Av
str39,str12,str12,float64,float64,float64,int64,int64,str12,float64,str7,float64,float64,float64,int64,int64,str12,float64,str7,float64,float64,float64,int64,int64,str12,float64,str7,float64,float64,float64,int64,int64,str12,float64,str7,float64,float64,float64,int64,int64,str12,float64,str7,float64,float64,float64,int64,str36,str12,float64,str42,str80,float64,float64,float64,float64
ZTF17aaajnki_20180320_P60_v1.ascii,ZTF17aaajnki,kcSB2,17.99,-0.0,0.0015,1,1,kcSB2,17.99,Gal,-0.0,0.0015,-99.9,2,2,kcSB1,17.3,Gal,0.0001,0.0016,-99.9,2,3,PS1-10bzj,10.61,Ic-SLSN,0.0003,0.0026,6.55,0,5,kcSB4,5.8,Gal,-0.0002,0.0041,-99.9,2,8,sn07it,5.28,II-norm,0.0007,0.0051,191.0,0,ZTF17aaajnki_20180320_P60_v1_10,IIn,0.001113,IIn/2010jl/KAST+2455651.6779977,/mnt/c/Users/20xha/Documents/Caltech/Research/superfit/bank/binnings/10A/gal/SB2,0.39013362,0.6098664,164.178,1.6


In [49]:
phase_data[0]

ZTFID,IAUID,RA,Dec,peakt,peakfilt,peakmag,peakabs,duration,rise,fade,type,redshift,b,A_V
str12,str10,str11,str11,float64,str1,float64,str6,str8,str8,str8,str10,str7,float64,float64
ZTF18aagstdc,SN2018apn,15:50:03.55,+42:05:18.5,210.86,g,17.2413,-18.39,>10.484,>3.96,6.524,SN Ia,0.03,50.84684165,0.055


In [45]:
ZTFid = np.unique(joined_table["ZTF_Name"])

In [39]:
counter = 0
for j in phase_data["ZTFID"][0:100]:
    counter += 1
    if(counter % 10 == 0):
        print(counter)
    index = np.where(joined_table["ZTF_Name"] == j)
    for i in joined_table[index]:
        sn_path = "/home/xhall/Documents/Superfit/bank/binnings/10A/sne/" + i["Match"][0:i["Match"].rfind("/")] + "/" + short_path_dict[i["Match"]]
        gal_path = basedir + "Superfit/" +i["Gal"][i["Gal"].rfind("bank")::]
        data = Table.read("/home/xhall/Documents/NewZTF/spectra_nonan/" + i["Version"], format = "ascii")
        
        nova = np.loadtxt(sn_path)
        host = np.loadtxt(gal_path)
        nova[:,1]=nova[:,1]/np.nanmedian(nova[:,1])
        host[:,1]=host[:,1]/np.nanmedian(host[:,1])
        
        lam = np.linspace(max(np.min(host[:,0]),np.min(nova[:,0])),min(np.max(host[:,0]),np.max(nova[:,0])), 10000)
        
        nova_int = interpolate.interp1d(nova[:,0] , nova[:,1] ,   bounds_error=False, fill_value='nan')

        host_int = interpolate.interp1d(host[:,0], host[:,1],   bounds_error=False, fill_value='nan')

        host_nova = i["Frac(SN)"]*nova_int(lam) + i["Frac(gal)"]*host_int(lam)
        
        spectra_name = i["Version"].split(".")[0]
        z_snid = i["z_snid"]
        
        source_folder = source + spectra_name
        files = np.sort(glob.glob(source_folder+"/*.dat"))
        
        files = np.sort(files)
        matches_files = files[0:len(files)-1]
        spectra = Table.read(files[-1], format = "ascii", names = ["wavelength", "flux"])
        match = Table.read(files[0], format = "ascii", names = ["redshifted_wavelength", "flux"])
        
        low, high = 5300, 5700

        x, y = spectra["wavelength"] / (1 + i["z_snid_1"]), spectra["flux"]
        xs, ys = data["col1"] / (1 + i["Superfit_z"]), data["col2"]
        ys = ys/np.median(ys[np.where((xs < high) & (xs > low))])
        xi, yi = match["redshifted_wavelength"] / (1 + i["z_snid_1"]), match["flux"]
        xsi, ysi = lam, host_nova
        ysi = ysi/np.median(ysi[np.where((xsi < high) & (xsi > low))])

        min_overlap = max(np.min(xsi), np.min(xi))
        max_overlap = min(np.max(xsi), np.max(xi))
        interp_grid = np.linspace(min_overlap, max_overlap, 10000)
        snid_interp = interpolate.interp1d(xi, yi)
        supy_interp = interpolate.interp1d(xsi, ysi)
        sep_t = np.max(np.abs((supy_interp(interp_grid)) - (snid_interp(interp_grid))))

        min_overlap_d = max(np.min(x), np.min(xs))
        max_overlap_d = min(np.max(x), np.max(xs))
        interp_grid_d = np.linspace(min_overlap_d, max_overlap_d, 10000)
        snid_interp_d = interpolate.interp1d(x, y)
        supy_interp_d = interpolate.interp1d(xs, ys)
        sep_d = np.max(np.abs((supy_interp_d(interp_grid_d)) - (snid_interp_d(interp_grid_d))))

        sep = max(sep_d, sep_t)/2

        t_max = phase_data[np.where(phase_data["ZTFID"] == i["ZTF_Name"])]["peakt"] + 2458000

        date_obs = i["Version"].split("_")[1]
        astropy_t_obs = Time(date_obs[0:4] + "-" + date_obs[4:6] + "-" + date_obs[6::])
        t_obs = astropy_t_obs.jd
        phase_obs_snid = (t_obs-t_max)/(1 + i["z_snid_1"])
        phase_obs_superfit = (t_obs-t_max)/(1 + i["Superfit_z"])

        fig, ax = plt.subplots(figsize=(8,4.5))

        ax.plot(xs,ys + sep,color='#32384D',label='New SN (SNID $\phi = {:.0f}$)(Superfit $\phi = {:.0f}$)'.format(phase_obs_snid[0], phase_obs_superfit[0]),alpha=0.5)
        ax.plot(x,y- sep,color='#32384D' ,alpha=0.5)
        ax.plot(xi,yi - sep,color='#217CA3',label='SNID template (z = {:.3f})($\phi = {:.0f}$)'.format(i["z_snid_1"], i["age_1"]), lw=3)
        ax.plot(xsi,ysi + sep,color='#E29930',label='Superfit template (z = {:.3f})($\phi = {:.0f}$)'.format(i["Superfit_z"], i["Phase"]), lw=3)

        ax.plot(x[-3],y[-3],color='#217CA3', lw=3)
        ax.set_xlabel(r'Rest Frame Wavelength ($\mathrm{\AA}$)', fontsize=17)
        ax.set_ylabel('Relative Flux', fontsize=17)
        ax.tick_params(which='both',labelsize=15)

        ax.grid(axis='x', color='0.7', ls=':')
        ax.xaxis.set_minor_locator(MultipleLocator(250))
        ax.set_yticklabels([])

        ax.text(0.565, 0.955, 'Superfit type: ',
                va='center',
                fontsize=15, transform=plt.gcf().transFigure)
        ax.text(0.745, 0.955, i["Superfit_c"],
                color='#E29930', weight='bold', va='center',
                fontsize=23, transform=plt.gcf().transFigure)

        ax.text(0.105, 0.955, 'SNID type: ',
                va='center',
                fontsize=15, transform=plt.gcf().transFigure)
        ax.text(0.245, 0.955, i["c_snid_1"],
                color='#217CA3', weight='bold', va='center',
                fontsize=23, transform=plt.gcf().transFigure)

        ax.set_xlim(min(np.min(x),np.min(xs)),max(np.max(x),np.max(xs)))

        ax.legend(fancybox=True)
        fig.subplots_adjust(left=0.055,right=0.99,top=0.925,bottom=0.145)
        
        fig.savefig('/home/xhall/Documents/NewZTF/sample_2020/BTSoutput/' + j + '_' + i["Version"] + '.png', dpi = 600)
        plt.close(fig)
        plt.close()

10
20
30
40
50
60
70
80
90
100


In [24]:
np.where(phase_data["ZTFID"] == i["ZTF_Name"])

(array([], dtype=int64),)

In [26]:
i["ZTF_Name"]

'ZTF17aabcpcp'

In [27]:
i["ZTF_Name"] in phase_data["ZTFID"]

False

In [28]:
i["ZTF_Name"]

'ZTF17aabcpcp'

In [31]:
counter

1

In [33]:
for j in phase_data["ZTFID"][0:100]:
    if(j not in joined_table["ZTF_Name"]):
        print(j)

ZTF18aaqjcvv
ZTF18aarbgdn
ZTF18aaphzut
ZTF18aahjdnz
ZTF18aakqtwt
ZTF18aaqfziz
ZTF18aamfrvy
ZTF18aaripcr
ZTF18aabdjmb
ZTF18aauuqrh
